In [0]:
%pip install -qqqq -U -r requirements.txt

In [0]:
dbutils.library.restartPython()

In [0]:
from cookbook.tools.vector_search import (
    VectorSearchRetrieverTool,
    VectorSearchSchema,
)
from cookbook.tools.uc_tool import UCTool
from cookbook.agents.utils.databricks_model_serving_client import DatabricksModelServingClient

In [0]:
retriever_tool = VectorSearchRetrieverTool(
    name="search_product_docs",
    description="Use this tool to search for product documentation.",
    vector_search_index="dbdemos.dbdemos_rag_chatbot.databricks_documentation_vs_index",
    vector_search_schema=VectorSearchSchema(
        # These columns are the default values used in the `01_data_pipeline` notebook
        # If you used a different column names in that notebook OR you are using a pre-built vector index, update the column names here.
        chunk_text="content",  # Contains the text of each document chunk
        document_uri="url",  # The document URI of the chunk e.g., "/Volumes/catalog/schema/volume/file.pdf" - displayed as the document ID in the Review App
        # additional_metadata_columns=[],  # Additional columns to return from the vector database and present to the LLM
    )
)


translate_sku_tool = UCTool(uc_function_name="devanshu_pandey.cmhc_demo.vector_index_search_tool")

tools = [retriever_tool]

In [0]:
entry_point = dbutils.notebook.entry_point

host_name = f'https://{entry_point.getDbutils().notebook().getContext().browserHostName().get()}'

token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

In [0]:
base_url = f"{host_name}/serving-endpoints/"
base_url

In [0]:
config_list = {
    "model_client_cls": "DatabricksModelServingClient",
    "model": "gpt4o",
    "endpoint_name": "databricks-meta-llama-3-3-70b-instruct",
    "llm_config": {"temperature": 0.5, "max_tokens": 1500}
    
}

In [0]:
def is_termination_message(message):
    content = message.get("content", "")
    return (content and "TERMINATE" in content.upper()) or (message['role'] == 'user' and 'tool_calls' not in message)

In [0]:
import os

from autogen import ConversableAgent

def create_agents(system_prompt, chat_history):
    # The user proxy agent is used for interacting with the assistant agent
    # and executes tool calls.
    user_proxy = ConversableAgent(
        name="User",
        llm_config=False,
        is_termination_msg=is_termination_message,
        human_input_mode="NEVER",
    )

    assistant = ConversableAgent(
        name="Assistant",
        system_message="You are a helpful AI assistant. "
        "You can help with simple calculations. "
        "Return 'TERMINATE' when the task is done.",
        llm_config={"config_list": [config_list]},
        chat_messages={user_proxy: chat_history}
    )

    return assistant, user_proxy
    
assistant, user_proxy = create_agents('test', [])

In [0]:
from autogen import register_function

for tool in tools:
  register_function(
      tool,
      caller=assistant,  # The assistant agent can suggest calls to the calculator.
      executor=user_proxy,  # The user proxy agent can execute the calculator calls.
      name=tool.name,
      description=tool.description,  # A description of the tool.
  )

In [0]:
translate_sku_tool._toolkit.tools[0].register_function(callers = assistant,
                                executors = user_proxy )

In [0]:
assistant.register_model_client(model_client_cls=DatabricksModelServingClient)

In [0]:
chat_result = user_proxy.initiate_chat(assistant, message="What is mlflow in databricks?")

In [0]:
assistant.last_message(user_proxy)

In [0]:
history = assistant.chat_messages[user_proxy]
history

In [0]:
assistant, user_proxy = create_agents('test', history)

In [0]:
assistant.chat_messages[user_proxy]

In [0]:
assistant.register_model_client(model_client_cls=DatabricksModelServingClient)

In [0]:
chat_result = user_proxy.initiate_chat(assistant, message="Can you summary our actual conversation?", clear_history=False)

In [0]:
assistant.last_message(user_proxy)

In [0]:
from autogen import GroupChat, GroupChatManager
from autogen import register_function
from cookbook.agents.genie_agent import GenieAgent

'''
config_list = {
    "model": "casaman-gpt4o",
    "base_url": base_url,
    "api_key": token,
}
'''

config_list = {
    "model_client_cls": "DatabricksModelServingClient",
    "model": "gpt4o",
    "endpoint_name": "casaman-gpt4o",
    "llm_config": {"temperature": 0.01, "max_tokens": 1500}
}

genie_agent = GenieAgent("Genie", genie_space_id="01ef92421847143785bb9e765454528e")

genie_agent.description = "You are a Genie Agent. Genie is a helpful AI Assistant that helps to answer questions about a database that contains structured data. It helps to answer questions about the metadata of the tables in the database and also answers questions about the data in the tables."

user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=is_termination_message,
    human_input_mode="NEVER",
)

assistant = ConversableAgent(
    name="Function_Tool_Agent",
    system_message="You are a helpful AI assistant. "
        "You can help with questions related with Databricks documentation.",
    llm_config={"config_list": [config_list], "cache_seed": None}
)

assistant.description = "You are a helful AI assistant. You can help with questions related with Databricks documentation."


for tool in tools:
  register_function(
      tool,
      caller=assistant,  # The assistant agent can suggest calls to the calculator.
      executor=user_proxy,  # The user proxy agent can execute the calculator calls.
      name=tool.name,
      description=tool.description,  # A description of the tool.
  )

system_message = "You are in a role play game. The following roles are available: {roles}. Read the following conversation. Then select the next role from {agentlist} to play or return TERMINATE if the task is solved. Only return the role."

last_message = "Read the above conversation. Then select the next role from {agentlist} to play or return TERMINATE if the task is solved. Only return the role."


group_chat = GroupChat([assistant, user_proxy, genie_agent], messages=[], max_round=2, speaker_selection_method="auto",
                       select_speaker_message_template=system_message,
                       select_speaker_prompt_template=last_message,
                       select_speaker_auto_model_client_cls=DatabricksModelServingClient,
                       select_speaker_auto_llm_config={"config_list": [config_list]} )

is_termination_msg = lambda x: "TERMINATE" in x.get("content", "")

group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": [config_list], "cache_seed": None},
    is_termination_msg=is_termination_msg
)

assistant.register_model_client(DatabricksModelServingClient)

In [0]:
chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message="Can you tell me what tables you have access to?"
)

In [0]:
chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message="what is mlflow in databricks?"
)

In [0]:
chat_result = user_proxy.initiate_chat(
    group_chat_manager,
    message="Do you know something about DLT in databricks?"
)

In [0]:
group_chat_manager.last_message(user_proxy)

In [0]:
group_chat_manager.chat_messages[user_proxy][-2]